In [25]:
import pandas as pd
import numpy as np
abs_path = "D:\stocknet-dataset-master\price\preprocessed"
df = pd.read_csv(abs_path + '\AAPL.txt', delimiter = "\t",
                names=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
# df.columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]

In [26]:
df = df.iloc[::-1]
df = df.reset_index(drop=True)

In [27]:
df = df[["Date","High","Low","Close"]]
df

,Date,High,Low,Close
0,2012-09-05,0.107768,0.109047,0.097979
1,2012-09-06,0.111639,0.120094,0.107725
2,2012-09-07,0.109697,0.116947,0.105966
3,2012-09-10,0.106800,0.111420,0.076953
4,2012-09-11,0.110742,0.119075,0.096363
...,...,...,...,...
1252,2017-08-28,0.001752,0.013387,0.000438
1253,2017-08-29,-0.008485,0.010219,-0.009104
1254,2017-08-30,0.005463,0.006016,-0.001842
1255,2017-08-31,0.001775,0.007163,0.000796


#### Extract timestamp

In [44]:
exportFolder = "../00 dataset/"

In [45]:
timestamp = pd.to_datetime(df['Date'])
timestamp.to_csv(exportFolder+'timestamp.csv',  header=True)

In [46]:
with open(exportFolder+'timestamp.npy', 'wb') as f:
    np.save(f, timestamp.to_numpy())

In [47]:
with open(exportFolder+'timestamp.npy', 'rb') as f:
    a = np.load(f)

In [48]:
a

array(['2012-09-05T00:00:00.000000000', '2012-09-06T00:00:00.000000000',
       '2012-09-07T00:00:00.000000000', ...,
       '2017-08-30T00:00:00.000000000', '2017-08-31T00:00:00.000000000',
       '2017-09-01T00:00:00.000000000'], dtype='datetime64[ns]')

#### Extract label

In [50]:
import numpy as np

# init column with derived value np.nan
df['Label'] = np.nan
df.head(2)

,Date,High,Low,Close,Label
0,2012-09-05,0.107768,0.109047,0.097979,NaN
1,2012-09-06,0.111639,0.120094,0.107725,NaN


In [51]:
# apply label fram index 5 ~ last
# if price ↑ then assign 1
for i in range(1, len(df)):
    df.loc[i, 'Label'] = 1 if df.loc[i, 'Close'] >= df.loc[i-1, 'Close'] else 0

In [52]:
df.head(10)

,Date,High,Low,Close,Label
0,2012-09-05,0.107768,0.109047,0.097979,NaN
1,2012-09-06,0.111639,0.120094,0.107725,1.0
2,2012-09-07,0.109697,0.116947,0.105966,0.0
3,2012-09-10,0.106800,0.111420,0.076953,0.0
4,2012-09-11,0.110742,0.119075,0.096363,1.0
5,2012-09-12,0.117272,0.122382,0.099094,1.0
6,2012-09-13,0.119309,0.132744,0.115013,1.0
7,2012-09-14,0.118095,0.129471,0.114741,0.0
8,2012-09-17,0.119705,0.120425,0.112116,0.0
9,2012-09-18,0.106942,0.110817,0.101470,0.0


In [53]:
df.tail(10)

,Date,High,Low,Close,Label
1247,2017-08-21,0.000000,0.002476,-0.015175,0.0
1248,2017-08-22,0.006488,0.017747,0.005152,1.0
1249,2017-08-23,-0.004444,0.004318,-0.005633,0.0
1250,2017-08-24,0.002813,0.004751,-0.008939,0.0
1251,2017-08-25,0.002386,0.008099,0.000000,1.0
1252,2017-08-28,0.001752,0.013387,0.000438,1.0
1253,2017-08-29,-0.008485,0.010219,-0.009104,0.0
1254,2017-08-30,0.005463,0.006016,-0.001842,1.0
1255,2017-08-31,0.001775,0.007163,0.000796,1.0
1256,2017-09-01,0.004878,0.005732,-0.002256,0.0


In [54]:
label = df['Label']
label.to_csv(exportFolder+'label.csv',  header=True)

In [55]:
with open(exportFolder+'label.npy', 'wb') as f:
    np.save(f, label.to_numpy())

In [56]:
with open(exportFolder+'label.npy', 'rb') as f:
    a = np.load(f)
a

array([nan,  1.,  0., ...,  1.,  1.,  0.])

#### Extract price data

In [58]:
price = df[["High","Low","Close"]]
price.to_csv(exportFolder+'price.csv',  header=True)

In [59]:
with open(exportFolder+'price.npy', 'wb') as f:
    np.save(f, price.to_numpy())

In [60]:
with open(exportFolder+'price.npy', 'rb') as f:
    a = np.load(f)
a

array([[ 0.107768,  0.109047,  0.097979],
       [ 0.111639,  0.120094,  0.107725],
       [ 0.109697,  0.116947,  0.105966],
       ...,
       [ 0.005463,  0.006016, -0.001842],
       [ 0.001775,  0.007163,  0.000796],
       [ 0.004878,  0.005732, -0.002256]])

#### Get all files of a directory

In [3]:
from os import walk

_, _, filenames = next(walk(abs_path))

In [18]:
print(len(filenames))
filenames[:5]

88


['AAPL.txt', 'ABB.txt', 'ABBV.txt', 'AEP.txt', 'AGFS.txt']